In [97]:
import pandas as pd
import numpy as np

In [3]:
invoice_df = pd.read_pickle("/Users/tigrangaplanyan/Downloads/data/invoices_new.pkl")

In [136]:
with open('/Users/tigrangaplanyan/Downloads/data/expired_invoices.txt', 'r') as file:
    # Read the content of the file
    content = file.read()

In [7]:
len(invoice_df)

100

In [56]:
invoice_df[0]

{'id': '365371O',
 'created_on': '2019-05-27',
 'items': [{'item': {'id': 142446,
    'name': 'ii_142446',
    'unit_price': 182,
    'type': 1},
   'quantity': 6},
  {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1},
   'quantity': 1}]}

In [71]:
invoice_df[2]

{'id': 341762,
 'created_on': '2019-06-19',
 'items': [{'item': {'id': 143176,
    'name': 'ii_143176',
    'unit_price': 175,
    'type': 2},
   'quantity': 2},
  {'item': {'id': 161765, 'name': 'ii_161765', 'unit_price': 170, 'type': 2},
   'quantity': 8},
  {'item': {'id': 167763, 'name': 'ii_167763', 'unit_price': 150, 'type': 3},
   'quantity': 5},
  {'item': {'id': 195203, 'name': 'ii_195203', 'unit_price': 194, 'type': 0},
   'quantity': 3},
  {'item': {'id': 188321, 'name': 'ii_188321', 'unit_price': 101, 'type': 3},
   'quantity': 1},
  {'item': {'id': 114990, 'name': 'ii_114990', 'unit_price': 132, 'type': 3},
   'quantity': 2},
  {'item': {'id': 126940, 'name': 'ii_126940', 'unit_price': 166, 'type': 0},
   'quantity': 3}]}

In [90]:
from datetime import datetime 

def is_valid_date(date_str, date_format="%Y-%m-%d"):
    try:
        datetime.strptime(date_str, date_format)
        return True
    except ValueError:
        return False

In [146]:
records = []
for invoice in invoice_df:
    if "items" in invoice:
        for item in invoice['items']:
            total_price = item["item"]["unit_price"] * item['quantity']
            if is_valid_date(invoice['created_on']):
                date = pd.to_datetime(invoice['created_on'])
            else:
                pass
            quantity = item["quantity"]
            records.append({
                'invoice_id': invoice['id'],
                'created_on': date,
                'invoiceitem_id': item["item"]["id"],
                'invoiceitem_name': item["item"]["name"],
                'type': item["item"]["type"],
                'unit_price': item["item"]["unit_price"],
                'total_price': total_price,
                'quantity': quantity
            })

In [147]:
print(records[0])

{'invoice_id': '365371O', 'created_on': Timestamp('2019-05-27 00:00:00'), 'invoiceitem_id': 142446, 'invoiceitem_name': 'ii_142446', 'type': 1, 'unit_price': 182, 'total_price': 1092, 'quantity': 6}


In [148]:
df = pd.DataFrame(records)

In [149]:
df

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,quantity
0,365371O,2019-05-27,142446,ii_142446,1,182,1092,6
1,365371O,2019-05-27,154019,ii_154019,1,176,176,1
2,389528,2019-03-19,172535,ii_172535,0,105,420,4
3,389528,2019-03-19,175208,ii_175208,0,109,763,7
4,389528,2019-03-19,139970,ii_139970,2,193,772,4
...,...,...,...,...,...,...,...,...
487,321243,2019-05-26,175528,ii_175528,1,169,507,3
488,377960,2019-02-22,123242,ii_123242,1,110,550,5
489,377960,2019-02-22,196386,ii_196386,3,150,900,6
490,377960,2019-02-22,196707,ii_196707,2,191,573,3


In [150]:
    df["type"] = np.where(df["type"] == 0, "Material", np.where(df["type"] == 1, "Equipment", np.where(df["type"] == 2, "Service", np.where(df["type"] == 3, "Other", "Invalid"))))

In [151]:
df

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,quantity
0,365371O,2019-05-27,142446,ii_142446,Equipment,182,1092,6
1,365371O,2019-05-27,154019,ii_154019,Equipment,176,176,1
2,389528,2019-03-19,172535,ii_172535,Material,105,420,4
3,389528,2019-03-19,175208,ii_175208,Material,109,763,7
4,389528,2019-03-19,139970,ii_139970,Service,193,772,4
...,...,...,...,...,...,...,...,...
487,321243,2019-05-26,175528,ii_175528,Equipment,169,507,3
488,377960,2019-02-22,123242,ii_123242,Equipment,110,550,5
489,377960,2019-02-22,196386,ii_196386,Other,150,900,6
490,377960,2019-02-22,196707,ii_196707,Service,191,573,3


In [152]:
df["quantity"].value_counts

<bound method IndexOpsMixin.value_counts of 0      6
1      1
2      4
3      7
4      4
      ..
487    3
488    5
489    6
490    3
491    7
Name: quantity, Length: 492, dtype: object>

In [153]:
df["quantity"] = pd.to_numeric(df["quantity"], errors = "coerce")

In [154]:
df["unit_price"] = pd.to_numeric(df["unit_price"], errors = "coerce")

In [155]:
df = df.dropna()

In [156]:
df.isna().sum()

invoice_id          0
created_on          0
invoiceitem_id      0
invoiceitem_name    0
type                0
unit_price          0
total_price         0
quantity            0
dtype: int64

In [157]:
df["invoice_total"] = df["quantity"] * df["unit_price"]

/var/folders/0c/j4c53d0102lcr2b5rk247yrm0000gn/T/ipykernel_1582/997830828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["invoice_total"] = df["quantity"] * df["unit_price"]


In [158]:
df

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,quantity,invoice_total
0,365371O,2019-05-27,142446,ii_142446,Equipment,182,1092,6.0,1092.0
1,365371O,2019-05-27,154019,ii_154019,Equipment,176,176,1.0,176.0
2,389528,2019-03-19,172535,ii_172535,Material,105,420,4.0,420.0
3,389528,2019-03-19,175208,ii_175208,Material,109,763,7.0,763.0
4,389528,2019-03-19,139970,ii_139970,Service,193,772,4.0,772.0
...,...,...,...,...,...,...,...,...,...
487,321243,2019-05-26,175528,ii_175528,Equipment,169,507,3.0,507.0
488,377960,2019-02-22,123242,ii_123242,Equipment,110,550,5.0,550.0
489,377960,2019-02-22,196386,ii_196386,Other,150,900,6.0,900.0
490,377960,2019-02-22,196707,ii_196707,Service,191,573,3.0,573.0


In [127]:
df["invoice_total"] = sum(df["invoice_total"])

/var/folders/0c/j4c53d0102lcr2b5rk247yrm0000gn/T/ipykernel_1582/3798134454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["invoice_total"] = sum(df["invoice_total"])


In [128]:
df

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,quantity,invoice_total,percentage_in_invoice
0,365371O,2019-05-27,142446,ii_142446,Equipment,182,1092,6.0,106962.0,1.0
1,365371O,2019-05-27,154019,ii_154019,Equipment,176,176,1.0,106962.0,1.0
2,389528,2019-03-19,172535,ii_172535,Material,105,420,4.0,106962.0,1.0
3,389528,2019-03-19,175208,ii_175208,Material,109,763,7.0,106962.0,1.0
4,389528,2019-03-19,139970,ii_139970,Service,193,772,4.0,106962.0,1.0
...,...,...,...,...,...,...,...,...,...,...
152,363284,2019-09-21,144611,ii_144611,Equipment,172,860,5.0,106962.0,1.0
153,373241,2019-02-19,196536,ii_196536,Other,196,1372,7.0,106962.0,1.0
154,373241,2019-02-19,161678,ii_161678,Service,140,840,6.0,106962.0,1.0
155,373241,2019-02-19,116519,ii_116519,Equipment,161,161,1.0,106962.0,1.0


In [129]:
df["percentage_in_invoice"] = df["total_price"] / df["invoice_total"]

/var/folders/0c/j4c53d0102lcr2b5rk247yrm0000gn/T/ipykernel_1582/2735430319.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["percentage_in_invoice"] = df["total_price"] / df["invoice_total"]


In [130]:
df

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,quantity,invoice_total,percentage_in_invoice
0,365371O,2019-05-27,142446,ii_142446,Equipment,182,1092,6.0,106962.0,0.010209
1,365371O,2019-05-27,154019,ii_154019,Equipment,176,176,1.0,106962.0,0.001645
2,389528,2019-03-19,172535,ii_172535,Material,105,420,4.0,106962.0,0.003927
3,389528,2019-03-19,175208,ii_175208,Material,109,763,7.0,106962.0,0.007133
4,389528,2019-03-19,139970,ii_139970,Service,193,772,4.0,106962.0,0.007218
...,...,...,...,...,...,...,...,...,...,...
152,363284,2019-09-21,144611,ii_144611,Equipment,172,860,5.0,106962.0,0.00804
153,373241,2019-02-19,196536,ii_196536,Other,196,1372,7.0,106962.0,0.012827
154,373241,2019-02-19,161678,ii_161678,Service,140,840,6.0,106962.0,0.007853
155,373241,2019-02-19,116519,ii_116519,Equipment,161,161,1.0,106962.0,0.001505


In [132]:
df = df.drop(["quantity", "invoice_total"], axis = 1)

In [133]:
df

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,percentage_in_invoice
0,365371O,2019-05-27,142446,ii_142446,Equipment,182,1092,0.010209
1,365371O,2019-05-27,154019,ii_154019,Equipment,176,176,0.001645
2,389528,2019-03-19,172535,ii_172535,Material,105,420,0.003927
3,389528,2019-03-19,175208,ii_175208,Material,109,763,0.007133
4,389528,2019-03-19,139970,ii_139970,Service,193,772,0.007218
...,...,...,...,...,...,...,...,...
152,363284,2019-09-21,144611,ii_144611,Equipment,172,860,0.00804
153,373241,2019-02-19,196536,ii_196536,Other,196,1372,0.012827
154,373241,2019-02-19,161678,ii_161678,Service,140,840,0.007853
155,373241,2019-02-19,116519,ii_116519,Equipment,161,161,0.001505


In [142]:
content

'305869, 377307, 391273, 385290, 331902, 397723, 343695, 340601, 347510, 325156, 379387, 348894, 322229, 326649, 377960, 325063, 369378, 319405, 383681, 367288, 356552, 394428, 381476, 381457, 349879, 343254, 366751, 330931, 363263, 326452, 379687, 356532, 383235, 313012, 368913, 352442, 315960, 351096, 331193, 392657, 352391, 389528, 338547, 379961, 337140, 323231, 371205, 378746, 307175, 340299'

In [143]:
content = content.split(",")

In [138]:
if df["invoice_id"] in content:
    df["is_expired"] = True
else:
    df["is_expired"] = False

TypeError: 'in <string>' requires string as left operand, not Series